Timing Rebuild


In [10]:
import torch
from typing import Callable, Any

In [11]:
torch.cuda.is_available()

True

In [12]:
def time_pytorch_function(func: Callable, input):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)

    # warmup
    for _ in range(5):
        func(input)

    start.record()
    func(input)
    end.record()
    torch.cuda.synchronize()
    return start.elapsed_time(end)

In [13]:
b = torch.randn(10000, 10000).cuda()

In [14]:
def square_2(a):
    return a * a


def square_3(a):
    return a**2

In [15]:
time_pytorch_function(torch.square, b)

3.066879987716675

In [16]:
time_pytorch_function(torch.square, square_2(b))

3.066879987716675

In [17]:
time_pytorch_function(torch.square, square_3(b))

3.066879987716675

In [23]:
print("=============")
print("Profiling torch.square")
print("=============")

with torch.profiler.profile() as prof:
    torch.square(b)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

Profiling torch.square
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                           aten::square         0.89%      46.000us        44.35%       2.305ms       2.305ms       0.000us         0.00%       3.055ms       3.055ms             1  
                                              aten::pow        24.82%       1.290ms        43.47%       2.259ms       2.259ms       3.055ms       100.00%       3.055ms       3.055ms   

STAGE:2025-03-23 10:29:17 473:473 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2025-03-23 10:29:17 473:473 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2025-03-23 10:29:17 473:473 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


In [24]:
print("=============")
print("Profiling a * a")
print("=============")

with torch.profiler.profile() as prof:
    square_2(b)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

Profiling a * a
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                              aten::mul        24.84%       1.322ms        46.23%       2.461ms       2.461ms       3.054ms       100.00%       3.054ms       3.054ms             1  
void at::native::vectorized_elementwise_kernel<4, at...         0.00%       0.000us         0.00%       0.000us       0.000us       3.054ms       100.00%       3.054ms       3.054ms          

STAGE:2025-03-23 10:29:22 473:473 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2025-03-23 10:29:22 473:473 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2025-03-23 10:29:22 473:473 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


In [25]:
print("=============")
print("Profiling a ** 2")
print("=============")

with torch.profiler.profile() as prof:
    square_3(b)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

Profiling a ** 2
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                              aten::pow         7.36%       1.518ms        76.43%      15.757ms      15.757ms       5.076ms       100.00%       5.076ms       5.076ms             1  
void at::native::vectorized_elementwise_kernel<4, at...         0.00%       0.000us         0.00%       0.000us       0.000us       5.076ms       100.00%       5.076ms       5.076ms         

STAGE:2025-03-23 10:34:04 473:473 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2025-03-23 10:34:04 473:473 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2025-03-23 10:34:04 473:473 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


In [27]:
import torch
from torch.profiler import profile, record_function, ProfilerActivity

In [ ]:
## Default way to use profiler
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
    for _ in range(10):
        a = torch.square(torch.randn(10000, 10000).cuda())
prof.export_chrome_trace("trace.json")

STAGE:2025-03-23 10:46:25 473:473 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2025-03-23 10:46:36 473:473 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2025-03-23 10:46:36 473:473 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


In [ ]:
def create_trace_dl(trace_path: str) -> None:
    import json
    import base64
    from IPython.display import HTML

    with open(trace_path, "r") as f:
        data = json.load(f)
        json_str = json.dumps(data)
    b64 = base64.b64encode(json_str.encode()).decode()

    # Create a download link
    href = f'<a href="data:text/json;base64,{b64}" download="data.json">Download JSON</a>'
    HTML(href)

In [ ]:
create_trace_dl("./trace.json")

In [35]:
import json
import base64
from IPython.display import HTML

with open("./trace.json", "r") as f:
    data = json.load(f)
    json_str = json.dumps(data)
b64 = base64.b64encode(json_str.encode()).decode()

# Create a download link
href = f'<a href="data:text/json;base64,{b64}" download="data.json">Download JSON</a>'
HTML(href)

In [30]:
!cat trace.json


{
  "schemaVersion": 1,
  "deviceProperties": [
    {
      "id": 0, "name": "Quadro P5000", "totalGlobalMem": 17055940608,
      "computeMajor": 6, "computeMinor": 1,
      "maxThreadsPerBlock": 1024, "maxThreadsPerMultiprocessor": 2048,
      "regsPerBlock": 65536, "regsPerMultiprocessor": 65536, "warpSize": 32,
      "sharedMemPerBlock": 49152, "sharedMemPerMultiprocessor": 98304,
      "numSms": 20, "sharedMemPerBlockOptin": 49152
    }
  ],
  "traceEvents": [
  {
    "ph": "X", "cat": "cpu_op", "name": "aten::randn", "pid": 473, "tid": 473,
    "ts": 1742726785771386, "dur": 1005298,
    "args": {
      "External id": 8193,"Ev Idx": 0
    }
  },
  {
    "ph": "X", "cat": "cpu_op", "name": "aten::empty", "pid": 473, "tid": 473,
    "ts": 1742726785771393, "dur": 39,
    "args": {
      "External id": 8194,"Ev Idx": 1
    }
  },
  {
    "ph": "X", "cat": "cpu_op", "name": "aten::normal_", "pid": 473, "tid": 473,
    "ts": 1742726785771436, "dur": 1005233,
    "args": {
      "Exter

In [ ]:
# ## Default way to use profiler
# with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
#     for _ in range(10):
#         a = torch.square(torch.randn(10000, 10000).cuda())

# prof.export_chrome_trace("trace.json")


## With warmup and skip
# https://pytorch.org/docs/stable/profiler.html


# Non-default profiler schedule allows user to turn profiler on and off
# on different iterations of the training loop;
# trace_handler is called every time a new trace becomes available
def trace_handler(prof):
    print(prof.key_averages().table(sort_by="self_cuda_time_total", row_limit=-1))
    prof.export_chrome_trace("/tmp/test_trace_" + str(prof.step_num) + ".json")


with torch.profiler.profile(
    activities=[
        torch.profiler.ProfilerActivity.CPU,
        torch.profiler.ProfilerActivity.CUDA,
    ],
    # In this example with wait=1, warmup=1, active=2, repeat=1,
    # profiler will skip the first step/iteration,
    # start warming up on the second, record
    # the third and the forth iterations,
    # after which the trace will become available
    # and on_trace_ready (when set) is called;
    # the cycle repeats starting with the next step
    schedule=torch.profiler.schedule(wait=1, warmup=1, active=2, repeat=1),
    on_trace_ready=trace_handler,
    # on_trace_ready=torch.profiler.tensorboard_trace_handler('./log')
    # used when outputting for tensorboard
) as p:
    for iter in range(10):
        torch.square(torch.randn(10000, 10000).cuda())
        # send a signal to the profiler that the next iteration has started
        p.step()

STAGE:2025-03-23 10:45:07 473:473 ActivityProfilerController.cpp:312] Completed Stage: Warm Up


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                            aten::copy_         0.01%     119.000us         5.56%     123.457ms      61.728ms     122.917ms        95.25%     122.917ms      61.459ms             2  
                       Memcpy HtoD (Pageable -> Device)         0.00%       0.000us         0.00%       0.000us       0.000us     122.917ms        95.25%     122.917ms      61.459ms             2  
         

STAGE:2025-03-23 10:45:09 473:473 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2025-03-23 10:45:09 473:473 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
